Finetune a Pretrained Hugginface model

In [ ]:
!pip install transformers
!pip install transformers[torch]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.9 MB/s eta 0:00:00


In [ ]:
#Import Libraries
import os

#Data manipulation
import pandas as pd
import numpy as np
import datasets
from sklearn.model_selection import train_test_split
import collections

import re
import seaborn as sns

#finetuning
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

#Evaluation
from datasets import load_metric, load_dataset
from sklearn.metrics import f1_score
from huggingface_hub import notebook_login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#login to huggingface
notebook_login()

Activate weight and Biases

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

Load The Datasets

In [ ]:
#load data
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/Nors/Noise/NDD.csv',sep = ',')

#check first 5

df


,Unnamed: 0,text,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,FileName,Year,label
0,1115946.0,En Sirene.,202.0,202_støj_skrig_lyd_larm,"['støj', 'skrig', 'lyd', 'larm', 'hørtes', 'lø...","['Hvad var det for en Lyd der Hørtes?', '— Hva...",støj - skrig - lyd - larm - hørtes - lød - lyd...,0.791364,False,1882_EK_SelvherskerenIDetSkjulteRigeEllerNihil...,1882.0,True
1,283212.0,— Sandhed.,577.0,577_sandhed_sandheden_rene_taler,"['sandhed', 'sandheden', 'rene', 'taler', 'løg...","['„Det er Sandhed.“', '— Det er Sandhed.', '„D...",sandhed - sandheden - rene - taler - løgn - si...,0.000000,False,1877_Borring_FaergemandensDatter_2.paragraphed...,1877.0,False
2,271001.0,— Susanne.,1316.0,1316_susanne_pige_ung_heude,"['susanne', 'pige', 'ung', 'heude', 'forlovede...","['den unge Pige.', 'den unge Pige.', 'den unge...",susanne - pige - ung - heude - forlovede - bøm...,0.000000,False,1894_Schandorph_VilhelmVangsStudenteraar.parag...,1894.0,False
3,465532.0,„Bevares.“,1339.0,1339_bevares_gud_ih_herre,"['bevares', 'gud', 'ih', 'herre', 'nei', 'vorh...","['„Nei, Herre!“', '„Nei, Herre.“', '— Nei, Her...",bevares - gud - ih - herre - nei - vorherre - ...,0.000000,False,1874_Karstens_GrevindeDanner_1.paragraphed.txt...,1874.0,False
4,755992.0,„Hvilket?“,326.0,326_hvilket_hvorledes_indtryk_hva,"['hvilket', 'hvorledes', 'indtryk', 'hva', 'sp...","['»Hvilket?«', '„Hvilket?“', '„Hvad, hvilket?“']",hvilket - hvorledes - indtryk - hva - spurgte ...,1.000000,True,1883_Brosboell_Krybskytten.paragraphed.txt_ocr...,1883.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2102,NaN,Som en Følge heraf arriverede Tante og Neveu i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2103,NaN,Som en Følge heraf arriverede Tante og Neveu i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2104,NaN,Sværmen blev staaende; helt henne ved Omdrejni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2105,NaN,Alligevel kunde man ikke undgaa at føle sig im...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
train_df = df
train_df['label'].value_counts()

label
False    1556
True      551
Name: count, dtype: int64

In [ ]:
# Specify the columns you want to keep
columns_to_keep = ['text', 'label']

# Drop all columns except the specified ones
train_df = train_df[columns_to_keep]

In [ ]:
for label in train_df['label'].tolist()[:10]:
  if label ==False:
    print(label)

False
False
False
False
False
False
False
False
False


In [ ]:
# Create a dictionary to map the values
value_map = {
    False: '0',
    True: '1',
}

# Apply the mapping to the column
train_df['label'] = train_df['label'].map(value_map)

<ipython-input-235-7d919d36736d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].map(value_map)


In [ ]:
train_df

,text,label
0,En Sirene.,1
1,— Sandhed.,0
2,— Susanne.,0
3,„Bevares.“,0
4,„Hvilket?“,0
...,...,...
2102,Som en Følge heraf arriverede Tante og Neveu i...,1
2103,Som en Følge heraf arriverede Tante og Neveu i...,1
2104,Sværmen blev staaende; helt henne ved Omdrejni...,1
2105,Alligevel kunde man ikke undgaa at føle sig im...,1


In [ ]:
#view info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107 entries, 0 to 2106
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2107 non-null   object
 1   label   2107 non-null   object
dtypes: object(2)
memory usage: 33.0+ KB


# Remove the rows with null values in the label and agreement columns of the train_df DataFrame

In [ ]:
# Remove rows with null values
train_df.dropna(subset=['label', 'text'], inplace=True)

<ipython-input-238-662a41dc8b53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset=['label', 'text'], inplace=True)


In [ ]:
# View info once again
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107 entries, 0 to 2106
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2107 non-null   object
 1   label   2107 non-null   object
dtypes: object(2)
memory usage: 33.0+ KB


**Danish Text Preprocessing**

In [ ]:
#danish text preproccessing
import re
import nltk
import string

nltk.download('punkt')
nltk.download("stopwords")

def preprocess_danish(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(text.maketrans("", "", string.punctuation))
    text = re.sub(r'[^a-zA-ZæøåÆØÅ]', ' ', text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords (optional)
    #nltk.download('stopwords')
    from nltk.corpus import stopwords
    #stop_words = set(stopwords.words("danish"))
    #tokens = [word for word in tokens if word not in stop_words]

    return (" ".join(tokens))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Apply Text Preprocessing**

In [ ]:
# Apply the preprocessing function to the 'text_column' using apply
print(train_df['text'][:2])
train_df['text'] = train_df['text'].apply(preprocess_danish)
print()
print(train_df['text'][:2])
train_df['text'] = train_df['text'].str.strip()


0    En Sirene.
1    — Sandhed.
Name: text, dtype: object

0    en sirene
1      sandhed
Name: text, dtype: object


<ipython-input-241-cb4412434d01>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text'] = train_df['text'].apply(preprocess_danish)
<ipython-input-241-cb4412434d01>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text'] = train_df['text'].str.strip()


## Revert to original Dataset

In [ ]:
train_df.drop(train_df.columns.difference(['text', 'label']), axis=1, inplace=True)

<ipython-input-244-23126ca4e757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(train_df.columns.difference(['text', 'label']), axis=1, inplace=True)


AS Noticed in EDA we will drop the rows with missing values

In [ ]:
#Drop null rows
train_df.dropna(axis=0,inplace=True)

#check for the presence of null values again
train_df.isna().sum()


<ipython-input-245-d92b47589fe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(axis=0,inplace=True)


text     0
label    0
dtype: int64

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

Split the Train the dataset into training(for learning ) and evaluation(For computing metrics to capture overfitting) sets

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split


df = train_df

# Split the DataFrame with stratification based on the 'label' column
df_85, df_15 = train_test_split(df, test_size=0.15, random_state=42, stratify=df['label'])

# Print the shapes of the resulting DataFrames to verify
print(f"85% of data: {df_85.shape}")
print(f"15% of data: {df_15.shape}")


85% of data: (1790, 2)
15% of data: (317, 2)


In [ ]:
# train , eval = train_test_split(train_df,train_size=0.8,stratify=train_df['label'])
train = df_85
eval = df_15


In [ ]:
# Function to check if the value contains more than one word
def more_than_one_word(text):
    return len(text.split()) > 1

# Apply the function and filter the DataFrame
train = train[train['text'].apply(more_than_one_word)]
eval = eval[eval['text'].apply(more_than_one_word)]

check train data

In [ ]:
train.head()

,text,label
2059,den gamle zigeunerske hvis blege rynkede ansig...,0
1807,kerenskij udtaler sig skeptisk i anarkisten,1
1489,hvor underligt frøken anna brogaard blegnede d...,0
1663,skurken mumlede clemens mere og mere forbauset,0
250,men alt som heste og vogne kom frem blev der m...,1


Check eval

In [ ]:
eval.head()

,text,label
1069,han havde ventet sig eil hyrdetime,0
977,knuds side men denne opdagelse skete endnu ikk...,0
1843,hun var ganske ung i det højeste en fjorten fe...,0
1907,hr vismann for han har nu saa længe søgt efter...,0
1522,herrerne som sov i salonen maatte trive i bord...,0


In [ ]:
eval['label'].value_counts()

label
0    233
1     83
Name: count, dtype: int64

In [ ]:
train['label'].value_counts()

label
0    1312
1     468
Name: count, dtype: int64

Check the shapes of both dataset

In [ ]:
print(f'Shape of training set: {train.shape}\nShape of evaluation set: {eval.shape}')

Shape of training set: (1780, 2)
Shape of evaluation set: (316, 2)


Save the splitted data to be loaded as huggingface dataset

In [ ]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/eval_subset.csv", index=False)

In [ ]:
datasets = load_dataset('csv',
                        data_files={'train':'/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/train_subset.csv',
                        'eval':'/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/eval_subset.csv'}, encoding = "utf-8")

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
print(datasets['train'])

Dataset({
    features: ['text', 'label'],
    num_rows: 1780
})


## Creating a Tokenizer instance

- The line of code below initializes a tokenizer for the "tweet_sentiments_analysis_roberta" model using the transformers library. The tokenizer is an essential component for processing text data and is used to convert input text into tokens that can be fed into the model for sentiment analysis.

In [ ]:
#Create a tokenizer instance
tokenizer=AutoTokenizer.from_pretrained('KennethEnevoldsen/dfm-sentence-encoder-large')


tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/395k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

512

## Functions to tokenize text and transform labels


In [ ]:
#function to tokenize text
def tokenize_data(data):
    return tokenizer(data['text'],padding='max_length',max_length=512, truncation=True)


#function to transform labels
def transfom_label(data):

    #extract label
    label=data['label']
    num=0
    #create contitions
    if label==0: #non noise
        num=0
    elif label==1: #noise
        num=1

    return {'labels':num}

## Tokenizer text and trasform original sentiment labels


In [ ]:
#The following columns will be removed after tokenization
remove_columns = ['label', 'text']

#Tokenize the text data
datasets=datasets.map(tokenize_data,batched=True)

#transform the labels
datasets=datasets.map(transfom_label,remove_columns=remove_columns)

Map:   0%|          | 0/1780 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

Map:   0%|          | 0/1780 [00:00<?, ? examples/s]

Map:   0%|          | 0/316 [00:00<?, ? examples/s]

## Extract the train and eval datasets from datasets

In [ ]:
#extract train datasets
train_dataset=datasets['train'].shuffle(seed=0)
#extract eval datasets
eval_dataset=datasets['eval'].shuffle(seed=0)

## Define Metrics Function

In [ ]:
# Define the function to compute F1-score
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"f1-score": f1}


## Set the training argumets


In [ ]:
#set the training arguments
trainargs=TrainingArguments('/content/drive/MyDrive/Nors/Noise/Models/dfm_ND',
                            num_train_epochs=20,
                            evaluation_strategy="epoch",
                            save_strategy='epoch',
                            load_best_model_at_end=True,
                            metric_for_best_model = 'f1-score',
                            push_to_hub=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Create an instance of the Model

In [ ]:
# Create an instance of the BERT model
model_name = 'KennethEnevoldsen/dfm-sentence-encoder-large'
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

#the model used in this code are:
#KennethEnevoldsen/dfm-sentence-encoder-large
#vesteinn/DanskBERT
#MiMe-MeMo/MeMo-BERT-03
#NbAiLab/nb-bert-base

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KennethEnevoldsen/dfm-sentence-encoder-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create trainer instance

In [ ]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=trainargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

## Start training

In [ ]:
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("BERT model:")
print("Evaluation results:", results)

## Push model to huggingface

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1718204313.58029a0bd48e.1107.8:   0%|          | 0.00/9.38k [00:00<?, ?B/s]

events.out.tfevents.1718207036.58029a0bd48e.1107.9:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yemen2016/dfm_ND/commit/db9f5a053f60d9287c1ec63c53df71bef12148fa', commit_message='End of training', commit_description='', oid='db9f5a053f60d9287c1ec63c53df71bef12148fa', pr_url=None, pr_revision=None, pr_num=None)

## Evaluate the model on testing set

In [ ]:
from transformers import pipeline
clf = pipeline("text-classification", '-------/nbBERT_ND')
# The following are the list of fine-tuned models stored on huggingface
#nbBERT_ND
#dfm_ND
#memo_ND
#danskbert_ND

config.json:   0%|          | 0.00/922 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
#load data
import pandas as pd

test_df=pd.read_csv('/content/drive/MyDrive/Nors/Noise/test_ND_final.csv',sep = ',')

#check first 5

test_df

,text,label
0,finske Soldater og kosakkisk Kavaleri paradere...,N
1,Klokkerne ringer til Aftenmesse.\n,N
2,"Hyrdestilens Pastelfarver og Rosenskær, og hvo...",O
3,"Der blev danset. Én havde taget en Violin med,...",N
4,"Og Kirkeklokkerne ringer ustandseligt, stærker...",N
...,...,...
158,"sidste Rustvogn, mens Skuddene knaldede, og Tr...",N
159,Inde fra Salen klang som Svar en vild Galop fr...,N
160,Op ad Storegaden kom et af de østerrigske Regi...,N
161,Dansen gik endnu livligt til den brusende Musi...,N


In [ ]:
# Create a dictionary to map the values
value_map = {
    'O': '0',
    'N': '1',
}

# Apply the mapping to the column
test_df['label'] = test_df['label'].map(value_map)

In [ ]:
test_df

,text,label
0,finske Soldater og kosakkisk Kavaleri paradere...,1
1,Klokkerne ringer til Aftenmesse.\n,1
2,"Hyrdestilens Pastelfarver og Rosenskær, og hvo...",0
3,"Der blev danset. Én havde taget en Violin med,...",1
4,"Og Kirkeklokkerne ringer ustandseligt, stærker...",1
...,...,...
158,"sidste Rustvogn, mens Skuddene knaldede, og Tr...",1
159,Inde fra Salen klang som Svar en vild Galop fr...,1
160,Op ad Storegaden kom et af de østerrigske Regi...,1
161,Dansen gik endnu livligt til den brusende Musi...,1


In [ ]:
# Using readlines()
texts = test_df['text'].tolist()
labels = test_df['label'].tolist()
count = 0
acutal_list = []
predicted_list= []
# Strips the newline character
for text,label in zip(texts, labels):
    count += 1
    # print("Line{}: {}".format(count, line.strip()))
    # print(label)
    sentence = text
    actual_sentiment = label
    acutal_list.append(actual_sentiment)
    new_text = preprocess_danish(sentence)
    answer = clf(new_text)
    predicted_sentiment = answer[0]['label'][6:]
    predicted_list.append(predicted_sentiment)
    print(str(predicted_sentiment) + "\t" + str(actual_sentiment))

1	1
1	1
1	0
1	1
1	1
1	1
0	0
1	1
0	1
1	1
0	0
0	1
1	1
1	1
1	1
1	0
0	0
1	0
1	1
1	1
1	1
1	1
0	1
0	0
1	1
1	0
1	1
1	1
1	1
1	1
1	1
1	1
0	1
1	1
0	1
0	1
1	1
0	1
1	1
1	1
0	1
1	0
1	1
1	1
1	1
1	1
1	1
0	0
1	1
1	0
1	1
1	1
1	0
0	0
1	1
1	1
1	1
1	1
1	1
0	0
1	1
1	1
1	1
0	0
1	1
1	1
1	1
0	0
1	0
1	1
1	1
0	1
1	1
0	1
1	1
1	1
0	1
1	1
1	1
0	1
1	0
0	0
1	1
1	1
1	1
1	1
1	1
0	1
1	1
0	0
1	1
1	1
0	0
1	1
0	0
1	1
1	1
1	1
1	1
1	0
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
0	0
1	1
1	1
0	0
1	1
1	0
0	0
1	0
0	0
1	1
0	0
1	1
1	1
0	0
1	1
1	1
1	1
1	1
0	0
1	1
1	1
1	0
1	1
1	0
1	1
1	0
1	1
1	0
0	0
1	0
1	0
1	0
0	0
1	1
0	0
0	1
0	1
1	1
1	1
1	0
1	0
0	0
1	0
1	1
0	1
1	1
1	1
1	1
0	1
1	1
1	1
1	1
1	1
1	1
1	1


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import numpy as np

# Assuming acutal_list and predicted_list are already defined
# Calculate the F1-score for each class
f1_scores = f1_score(acutal_list, predicted_list, average=None)

# Calculate the weighted F1-score
weighted_f1_score = np.average(f1_scores, weights=np.bincount(acutal_list))

# Calculate the confusion matrix
conf_matrix = confusion_matrix(acutal_list, predicted_list)

# Calculate precision and recall for each class
precision = precision_score(acutal_list, predicted_list, average=None)
recall = recall_score(acutal_list, predicted_list, average=None)

print("F1-scores for each class:", f1_scores)
print("Weighted F1-score:", weighted_f1_score)
print("Confusion Matrix:\n", conf_matrix)
print("Precision for each class:", precision)
print("Recall for each class:", recall)


F1-scores for each class: [0.55172414 0.83682008]
Weighted F1-score: 0.7563635591142488
Confusion Matrix:
 [[ 24  22]
 [ 17 100]]
Precision for each class: [0.58536585 0.81967213]
Recall for each class: [0.52173913 0.85470085]
